In [25]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

In [26]:
# basic custom layout properties
width = 800
height = 800
x_width = 500
y_width = x_width
z_width = 400
z_aspect = 1.*(z_width/x_width)

In [ ]:
plotly_template = 'plotly_white'
color = 'black'

In [27]:
# define custom layout
        # plotly scene, layout, and camera
camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0., y=-.8, z=1.5),
    projection = dict(type = 'perspective')
) 

scene = dict(
    camera = camera,
    aspectmode = 'manual',
    aspectratio = dict(x=1, y=1, z=z_aspect),
    xaxis=dict(
        title='X\' (pc)', 
        range = [-x_width, x_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = color,
        linewidth = 2.,
    ), 
    yaxis=dict(
        title='Y\' (pc)', 
        range = [-y_width, y_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = color,
        linewidth = 2.,
    ), 
    zaxis=dict(
        title='Z\' (pc)', 
        range = [-z_width, z_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        showspikes = False,
        mirror = True,
        nticks = 5,
        linecolor = color,
        linewidth = 2.,
    )
)

legend = dict(
    x = 0,
    y = 1,
    title = dict(
        text = 'Click to toggle traces on/off',
        font = dict(
            size = 18,  
            color = color
        ),  
        side = 'top'
    ),
    font = dict(
        size = 14,
        family = 'Arial',
        color = color
    ),
    itemsizing = 'constant',
    bgcolor = 'rgba(0,0,0,0)' # transparent legend
)

layout = dict(
    scene = scene,
    template = 'plotly_dark', 
    dragmode = 'turntable',
    legend = legend,
    width = width,
    height = height
)

In [28]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=4.,
                #color='gray',
                color=dfv['extinction'].values[::ds_index],
                colorscale='gray',
                cmin = 0.0007,
                cmax = 0.005,
                symbol='circle',
                opacity=.04),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0


hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.03),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [29]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars', 'mass_50']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
ap = ap.merge(df_hunt_partIII_to_merge, on='name', how='left')

ap_reference_frame = ap[['x', 'y', 'z', 'U', 'V', 'W']].mean().values
ap_open_clusters = ap.loc[ap['kind'] == 'o']
ap_moving_groups = ap.loc[(ap['kind'] == 'm') | (ap['kind'].isna())]

In [30]:
save_name = '/Users/cam/Downloads/alphaPer_3d_plot.html'
plot_3d = StarClusters3DPlotter()
ap_oc_data = StarClusterData(ap_open_clusters, data_name = 'OCs', min_size = 0, max_size = 10, color = 'violet', opacity = 1)
ap_mg_data = StarClusterData(ap_moving_groups, data_name = 'MGs', min_size = 0, max_size = 5, color = 'violet', opacity = 1.)
sc_m6 = StarClusterCollection([ap_oc_data, ap_mg_data])
sc_m6.integrate_all_orbits(time=np.arange(0., -61., -1.), reference_frame_center=ap_reference_frame)
fig3d_m6 = plot_3d.generate_3d_plot(collection = sc_m6, show = False, save_name =save_name, figure_layout=layout, static_traces = static_traces, static_traces_times = static_traces_times, reference_frame_center = ap_reference_frame)